In [1]:

import pandas as pd
import numpy as np
import pickle
import scipy as spy
import requests
import seaborn as sns
from modules import data_collection

In [2]:
json = data_collection.FPL_Data_Collection().get_full_json()

In [3]:
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

## EDA

In [4]:
elements_df = pd.DataFrame(json['elements']).convert_dtypes()
element_types_df = pd.DataFrame(json['element_types']).convert_dtypes()
teams_df = pd.DataFrame(json['teams']).convert_dtypes()

In [5]:
elements_df.loc[elements_df.second_name == 'Haaland']

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
423,100,100,223094,0,0,7,-7,5,4,2.0,...,3,1,263,35,2,1,1,1,1.04247,0.43436


### Element Types

In [6]:
element_types_df.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],70
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],234
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],286
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],77


### Elements

In [7]:
elements_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,100,100,58822,0,0,-3,3,0,2,1.0,...,544,157,266,82,376,141,402,148,0.0,0.0
1,<NA>,<NA>,84450,0,0,1,-1,1,3,2.5,...,171,99,125,63,42,20,83,28,1.08062,0.46312
2,100,100,153256,0,0,-3,3,1,3,1.5,...,551,284,184,98,343,150,209,69,0.95745,0.0
3,<NA>,<NA>,156074,0,0,-3,3,0,2,1.0,...,552,163,268,84,379,144,419,155,0.0,0.0
4,100,100,167199,0,0,-2,2,0,3,3.0,...,260,157,97,49,73,35,245,80,1.03233,0.65693


In [8]:
elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

In [9]:
elements_df[['first_name','second_name','now_cost']]

,first_name,second_name,now_cost
0,Cédric,Alves Soares,42
1,Granit,Xhaka,51
2,Mohamed,Elneny,42
3,Rob,Holding,42
4,Thomas,Partey,48
...,...,...,...
662,Diego,Da Silva Costa,55
663,Boubacar,Traoré,45
664,Nathan,Fraser,45
665,Harvey,Griffiths,45


In [10]:
elements_df.describe().T

,count,mean,std,min,25%,50%,75%,max
chance_of_playing_next_round,405.0,65.802469,45.736931,0.0,0.0,100.0,100.0,100.0
chance_of_playing_this_round,402.0,56.716418,48.592808,0.0,0.0,100.0,100.0,100.0
code,667.0,227334.731634,149621.707974,11974.0,109389.5,198849.0,245621.5,601975.0
cost_change_event,667.0,-0.010495,0.139317,-1.0,0.0,0.0,0.0,1.0
cost_change_event_fall,667.0,0.010495,0.139317,-1.0,0.0,0.0,0.0,1.0
cost_change_start,667.0,-1.08096,1.704802,-6.0,-2.0,-1.0,0.0,9.0
cost_change_start_fall,667.0,1.08096,1.704802,-9.0,0.0,1.0,2.0,6.0
dreamteam_count,667.0,0.263868,0.609681,0.0,0.0,0.0,0.0,5.0
element_type,667.0,2.554723,0.82995,1.0,2.0,3.0,3.0,4.0
event_points,667.0,1.230885,2.407584,-1.0,0.0,0.0,1.0,16.0


In [11]:
elements_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 88 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   chance_of_playing_next_round          405 non-null    Int64  
 1   chance_of_playing_this_round          402 non-null    Int64  
 2   code                                  667 non-null    Int64  
 3   cost_change_event                     667 non-null    Int64  
 4   cost_change_event_fall                667 non-null    Int64  
 5   cost_change_start                     667 non-null    Int64  
 6   cost_change_start_fall                667 non-null    Int64  
 7   dreamteam_count                       667 non-null    Int64  
 8   element_type                          667 non-null    Int64  
 9   ep_next                               667 non-null    string 
 10  ep_this                               667 non-null    string 
 11  event_points       

### Teams

In [12]:
teams_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1200,1270,1160,1230,1160,1240,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1090,1100,1110,1110,1070,1090,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1050,1070,1070,1110,1040,1070,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1100,1150,1100,1150,1140,1170,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1140,1140,1170,1160,1150,1170,131


In [13]:
teams_df.columns

Index(['code', 'draw', 'form', 'id', 'loss', 'name', 'played', 'points',
       'position', 'short_name', 'strength', 'team_division', 'unavailable',
       'win', 'strength_overall_home', 'strength_overall_away',
       'strength_attack_home', 'strength_attack_away', 'strength_defence_home',
       'strength_defence_away', 'pulse_id'],
      dtype='object')

In [14]:
teams_df.describe().T

,count,mean,std,min,25%,50%,75%,max
code,20.0,25.75,27.332023,1.0,6.75,15.5,36.75,94.0
draw,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id,20.0,10.5,5.91608,1.0,5.75,10.5,15.25,20.0
loss,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
played,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
points,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
position,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
strength,20.0,3.25,0.786398,2.0,3.0,3.0,3.25,5.0
win,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
strength_overall_home,20.0,1135.25,83.21619,1045.0,1090.0,1100.0,1162.5,1350.0


In [15]:
teams_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   code                   20 non-null     Int64  
 1   draw                   20 non-null     Int64  
 2   form                   0 non-null      object 
 3   id                     20 non-null     Int64  
 4   loss                   20 non-null     Int64  
 5   name                   20 non-null     string 
 6   played                 20 non-null     Int64  
 7   points                 20 non-null     Int64  
 8   position               20 non-null     Int64  
 9   short_name             20 non-null     string 
 10  strength               20 non-null     Int64  
 11  team_division          0 non-null      object 
 12  unavailable            20 non-null     boolean
 13  win                    20 non-null     Int64  
 14  strength_overall_home  20 non-null     Int64  
 15  strength

## Cleaning

In [44]:
elements_df.columns.to_list()

['chance_of_playing_next_round',
 'chance_of_playing_this_round',
 'code',
 'cost_change_event',
 'cost_change_event_fall',
 'cost_change_start',
 'cost_change_start_fall',
 'dreamteam_count',
 'element_type',
 'ep_next',
 'ep_this',
 'event_points',
 'first_name',
 'form',
 'id',
 'in_dreamteam',
 'news',
 'news_added',
 'now_cost',
 'photo',
 'points_per_game',
 'second_name',
 'selected_by_percent',
 'special',
 'squad_number',
 'status',
 'team',
 'team_code',
 'total_points',
 'transfers_in',
 'transfers_in_event',
 'transfers_out',
 'transfers_out_event',
 'value_form',
 'value_season',
 'web_name',
 'minutes',
 'goals_scored',
 'assists',
 'clean_sheets',
 'goals_conceded',
 'own_goals',
 'penalties_saved',
 'penalties_missed',
 'yellow_cards',
 'red_cards',
 'saves',
 'bonus',
 'bps',
 'influence',
 'creativity',
 'threat',
 'ict_index',
 'starts',
 'expected_goals',
 'expected_assists',
 'expected_goal_involvements',
 'expected_goals_conceded',
 'influence_rank',
 'influence_r

In [16]:
elements_df.T[elements_df.dtypes == 'string']

,0,1,2,3,4,5,6,7,8,9,...,657,658,659,660,661,662,663,664,665,666
ep_next,1.0,2.5,1.5,1.0,3.0,10.0,1.0,0.0,7.0,0.5,...,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,1.3
ep_this,1.0,2.5,1.5,1.0,3.0,10.0,1.0,0.0,7.0,0.5,...,-0.5,-0.5,1.5,0.5,0.0,0.0,0.5,-0.5,-0.5,0.8
first_name,Cédric,Granit,Mohamed,Rob,Thomas,Martin,Kieran,Nicolas,Benjamin,Eddie,...,Connor,Chem,Gonçalo Manuel,Matheus Luiz,Sasa,Diego,Boubacar,Nathan,Harvey,Dexter
form,0.5,2.0,1.0,0.5,2.5,9.5,0.5,0.0,6.5,0.0,...,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
news,,,,,,,,Season-long loan to Nice,,,...,,,,,Knee injury - Expected back 01 Jun,,,,,
news_added,2022-09-16T13:30:06.551108Z,<NA>,2022-08-30T11:30:06.278675Z,<NA>,2022-09-25T09:00:06.484502Z,2022-09-18T20:00:06.315087Z,2022-09-26T13:00:06.430643Z,2022-08-26T07:24:29.541862Z,<NA>,<NA>,...,<NA>,<NA>,<NA>,2022-10-29T17:30:07.579364Z,2022-09-03T18:00:06.993294Z,2022-10-28T15:00:08.264065Z,2022-10-11T01:00:07.189826Z,<NA>,<NA>,<NA>
photo,58822.jpg,84450.jpg,153256.jpg,156074.jpg,167199.jpg,184029.jpg,192895.jpg,195735.jpg,198869.jpg,205533.jpg,...,198842.jpg,461026.jpg,181284.jpg,465351.jpg,429414.jpg,18507.jpg,476502.jpg,531363.jpg,449133.jpg,492776.jpg
points_per_game,1.0,4.4,1.3,1.0,3.8,5.9,1.6,0.0,4.7,1.0,...,1.0,1.0,2.3,2.4,1.0,1.2,1.5,0.0,0.0,1.0
second_name,Alves Soares,Xhaka,Elneny,Holding,Partey,Ødegaard,Tierney,Pépé,White,Nketiah,...,Ronan,Campbell,Ganchinho Guedes,Nunes,Kalajdzic,Da Silva Costa,Traoré,Fraser,Griffiths,Lembikisa
selected_by_percent,0.1,3.7,0.7,0.1,0.5,14.2,0.8,0.1,10.6,0.4,...,0.1,0.4,0.2,0.1,0.0,0.3,0.1,0.0,0.0,0.0


In [17]:
slim_elements_df = elements_df.copy()
slim_elements_df.drop(['first_name','second_name','photo','corners_and_indirect_freekicks_text','direct_freekicks_text','penalties_text'], axis =1, inplace=True)
slim_elements_df

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,100,100,58822,0,0,-3,3,0,2,1.0,...,544,157,266,82,376,141,402,148,0.0,0.0
1,<NA>,<NA>,84450,0,0,1,-1,1,3,2.5,...,171,99,125,63,42,20,83,28,1.08062,0.46312
2,100,100,153256,0,0,-3,3,1,3,1.5,...,551,284,184,98,343,150,209,69,0.95745,0.0
3,<NA>,<NA>,156074,0,0,-3,3,0,2,1.0,...,552,163,268,84,379,144,419,155,0.0,0.0
4,100,100,167199,0,0,-2,2,0,3,3.0,...,260,157,97,49,73,35,245,80,1.03233,0.65693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,100,0,18507,0,0,0,0,0,4,0.0,...,102,38,633,64,360,48,316,49,1.09223,0.21845
663,100,100,476502,0,0,0,0,0,3,1.0,...,392,213,205,102,326,142,449,168,0.94737,0.0
664,<NA>,<NA>,531363,0,0,0,0,0,4,0.0,...,417,72,558,58,629,72,555,69,0.0,0.0
665,<NA>,<NA>,449133,0,0,0,0,0,3,0.0,...,382,209,529,198,603,243,643,265,0.0,0.0


In [18]:
for col in slim_elements_df:
        if slim_elements_df[col].dtype != 'string':
            continue
        try:
            slim_elements_df[col] = slim_elements_df[col].astype('float')
        except ValueError as e:
            print(f'{col}: {e}')

news: could not convert string to float: ''
news_added: could not convert string to float: '2022-09-16T13:30:06.551108Z'
status: could not convert string to float: 'a'
web_name: could not convert string to float: 'Cédric'


In [19]:
slim_elements_df.T[slim_elements_df.dtypes == 'string']

,0,1,2,3,4,5,6,7,8,9,...,657,658,659,660,661,662,663,664,665,666
news,,,,,,,,Season-long loan to Nice,,,...,,,,,Knee injury - Expected back 01 Jun,,,,,
news_added,2022-09-16T13:30:06.551108Z,<NA>,2022-08-30T11:30:06.278675Z,<NA>,2022-09-25T09:00:06.484502Z,2022-09-18T20:00:06.315087Z,2022-09-26T13:00:06.430643Z,2022-08-26T07:24:29.541862Z,<NA>,<NA>,...,<NA>,<NA>,<NA>,2022-10-29T17:30:07.579364Z,2022-09-03T18:00:06.993294Z,2022-10-28T15:00:08.264065Z,2022-10-11T01:00:07.189826Z,<NA>,<NA>,<NA>
status,a,a,a,a,a,a,a,u,a,a,...,a,a,a,a,i,a,a,a,a,a
web_name,Cédric,Xhaka,Elneny,Holding,Partey,Ødegaard,Tierney,Pépé,White,Nketiah,...,Ronan,Campbell,Guedes,Matheus,Kalajdžić,Diego Costa,Traoré,Fraser,Griffiths,Lembikisa


In [20]:
slim_elements_df.expected_goals_per_90.value_counts()

0.0        310
0.05839      2
0.33626      1
0.00576      1
0.20678      1
          ... 
0.38552      1
0.01649      1
0.22253      1
0.0656       1
0.01549      1
Name: expected_goals_per_90, Length: 357, dtype: Int64

In [21]:
slim_elements_df.assign(start_cost = lambda x : x.now_cost - x.cost_change_start)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,start_cost
0,100,100,58822,0,0,-3,3,0,2,1.0,...,157,266,82,376,141,402,148,0.0,0.0,45
1,<NA>,<NA>,84450,0,0,1,-1,1,3,2.5,...,99,125,63,42,20,83,28,1.08062,0.46312,50
2,100,100,153256,0,0,-3,3,1,3,1.5,...,284,184,98,343,150,209,69,0.95745,0.0,45
3,<NA>,<NA>,156074,0,0,-3,3,0,2,1.0,...,163,268,84,379,144,419,155,0.0,0.0,45
4,100,100,167199,0,0,-2,2,0,3,3.0,...,157,97,49,73,35,245,80,1.03233,0.65693,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,100,0,18507,0,0,0,0,0,4,0.0,...,38,633,64,360,48,316,49,1.09223,0.21845,55
663,100,100,476502,0,0,0,0,0,3,1.0,...,213,205,102,326,142,449,168,0.94737,0.0,45
664,<NA>,<NA>,531363,0,0,0,0,0,4,0.0,...,72,558,58,629,72,555,69,0.0,0.0,45
665,<NA>,<NA>,449133,0,0,0,0,0,3,0.0,...,209,529,198,603,243,643,265,0.0,0.0,45


In [22]:
slim_elements_df.status = slim_elements_df.status.map({'i' : 'injury', 'u' : 'unavailble', 'd' : 'decision', 'a' : 'available'})

In [23]:
import _pickle as cPickle
with open(r"elements.pickle", "wb") as output_file:
    cPickle.dump(slim_elements_df, output_file)

In [24]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import xgboost as xgb
from scipy.stats import uniform, randint

In [25]:
X = slim_elements_df[['influence','creativity','threat','expected_goals_per_90','expected_assists_per_90','starts_per_90']]
y = slim_elements_df.total_points

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3791)

In [27]:
X_train.dtypes

influence                  float64
creativity                 float64
threat                     float64
expected_goals_per_90      Float64
expected_assists_per_90    Float64
starts_per_90              Float64
dtype: object

In [28]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")



def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))



In [29]:
xgb_model = xgb.XGBRegressor()

params = {
    'colsample_bytree' : uniform(0.7, 0.3),
    'gamma' : uniform(0, 0.5),
    'learning_rate' : uniform(0.03, 0.3),
    'max_depth' : randint(2, 6),
    'n_estimators' : randint(100, 150),
    'subsample' : uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=3791, n_iter=200, cv=3, verbose=1, return_train_score=True)

search.fit(X_train, y_train)

report_best_scores(search.cv_results_, 1)

"""
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
"""

Fitting 3 folds for each of 200 candidates, totalling 600 fits
Model with rank: 1
Mean validation score: 0.913 (std: 0.009)
Parameters: {'colsample_bytree': 0.7211153621711471, 'gamma': 0.059223005411420615, 'learning_rate': 0.09064203218319923, 'max_depth': 3, 'n_estimators': 122, 'subsample': 0.9133790808098412}



'\nxgb_model.fit(X_train, y_train)\ny_pred = xgb_model.predict(X_test)\nmse = mean_squared_error(y_test, y_pred)\n'

In [30]:
y_pred = search.best_estimator_.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse


45.92709305621939

In [31]:
r2 = r2_score(y_test, y_pred)
r2

0.9059859516010019

In [32]:
slim_elements_df.element_type.value_counts()

3    286
2    234
4     77
1     70
Name: element_type, dtype: Int64

In [33]:
check = slim_elements_df[['id','web_name','element_type','now_cost']]

In [34]:
check['y_test'] = y_test
check.dropna(how='any', inplace=True)
check['y_pred'] = y_pred

/tmp/ipykernel_11190/2138216554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['y_test'] = y_test
/tmp/ipykernel_11190/2138216554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check.dropna(how='any', inplace=True)
/tmp/ipykernel_11190/2138216554.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['y_p

In [35]:
check.sort_values('y_test', ascending=False).head(11)

,id,web_name,element_type,now_cost,y_test,y_pred
96,80,Toney,4,74,83,17.166922
137,111,Trossard,3,71,80,7.238916
23,26,Saliba,2,53,74,42.096783
14,16,Gabriel,2,52,72,35.809341
497,376,Pope,1,53,70,42.842049
376,279,Firmino,4,81,69,30.624594
523,398,Henderson,1,47,67,49.047649
378,281,Alisson,1,55,65,0.082976
445,335,Rashford,3,67,62,53.282578
1,3,Xhaka,3,51,62,51.041420


In [36]:

check.loc[check.y_test != 0].sort_values('y_test', ascending=False).head(20)


,id,web_name,element_type,now_cost,y_test,y_pred
96,80,Toney,4,74,83,17.166922
137,111,Trossard,3,71,80,7.238916
23,26,Saliba,2,53,74,42.096783
14,16,Gabriel,2,52,72,35.809341
497,376,Pope,1,53,70,42.842049
376,279,Firmino,4,81,69,30.624594
523,398,Henderson,1,47,67,49.047649
378,281,Alisson,1,55,65,0.082976
1,3,Xhaka,3,51,62,51.041420
445,335,Rashford,3,67,62,53.282578


In [37]:
full_y_pred = search.best_estimator_.predict(X)

In [38]:
slim_elements_df.team

0       1
1       1
2       1
3       1
4       1
       ..
662    20
663    20
664    20
665    20
666    20
Name: team, Length: 667, dtype: Int64

In [39]:
full_test = slim_elements_df[['id','web_name','team','element_type','now_cost']]
full_test['predict_points'] = full_y_pred
full_test['actual_points'] = y

/tmp/ipykernel_11190/3857854741.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_test['predict_points'] = full_y_pred
/tmp/ipykernel_11190/3857854741.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_test['actual_points'] = y


In [40]:
full_test

,id,web_name,team,element_type,now_cost,predict_points,actual_points
0,1,Cédric,1,2,42,1.658569,2
1,3,Xhaka,1,3,51,59.257935,62
2,4,Elneny,1,3,42,6.226771,4
3,5,Holding,1,2,42,1.271105,4
4,6,Partey,1,3,48,40.345520,42
...,...,...,...,...,...,...,...
662,625,Diego Costa,20,4,55,10.143930,7
663,629,Traoré,20,3,45,10.171195,12
664,657,Fraser,20,4,45,0.082976,0
665,658,Griffiths,20,3,45,0.082976,0


In [41]:
with open(r"../Data/test_optimize.pickle", "wb") as output_file:
    cPickle.dump(full_test, output_file)